# Building Agentic AI Applications In Snowflake

### *NOTE: For overview, prerequisites and other instructions, refer to this [guide](https://github.com/Snowflake-Labs/sfguide-build-data-agents-using-snowflake-cortex-ai).*


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Setup Tools

### Cortex Search: Tool for Unstructured Data

Setup a tool that will help the agent to extract information from unstructured data. It will process PDF documents about bikes and skis, and also use image descriptions. The information is stored in PDF and JPEG format.

### PDFs

In [ ]:
-- Preview documents
SELECT * FROM DIRECTORY('@DOCS');

Read/process the PDF files using SNOWFLAKE.CORTEX.PARSE_DOCUMENT

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE RAW_TEXT AS
SELECT 
    RELATIVE_PATH,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            '@DOCS',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'} ):content
        ) AS EXTRACTED_LAYOUT 
FROM 
    DIRECTORY('@DOCS')
WHERE
    RELATIVE_PATH LIKE '%.pdf';

In [ ]:
select * from RAW_TEXT limit 5;

Create the table that will be used by Cortex Search service as a tool for Cortex Agents in order to retrieve information from PDF and JPEG files

In [ ]:
create or replace TABLE DOCS_CHUNKS_TABLE ( 
    RELATIVE_PATH VARCHAR(16777216), -- Relative path to the PDF file
    CHUNK VARCHAR(16777216), -- Piece of text
    CHUNK_INDEX INTEGER, -- Index for the text
    CATEGORY VARCHAR(16777216) -- Will hold the document category to enable filtering
);


In [ ]:
insert into DOCS_CHUNKS_TABLE (relative_path, chunk, chunk_index)

    select relative_path, 
            c.value::TEXT as chunk,
            c.INDEX::INTEGER as chunk_index
            
    from 
        raw_text,
        LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
              EXTRACTED_LAYOUT,
              'markdown',
              1512,
              256,
              ['\n\n', '\n', ' ', '']
           )) c;

In [ ]:
SELECT * FROM DOCS_CHUNKS_TABLE limit 7;


Let's see how CLASSIFY_TEXT Cortex function can be used to classify the document type. We have two classes, Bike and Snow, and we pass the document title and the first chunk of the document to the function.

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE docs_categories AS WITH unique_documents AS (
  SELECT
    DISTINCT relative_path, chunk
  FROM
    docs_chunks_table
  WHERE 
    chunk_index = 0
  ),
 docs_category_cte AS (
  SELECT
    relative_path,
    TRIM(snowflake.cortex.CLASSIFY_TEXT (
      'Title:' || relative_path || 'Content:' || chunk, ['Bike', 'Snow']
     )['label'], '"') AS category
  FROM
    unique_documents
)
SELECT
  *
FROM
  docs_category_cte;

In [ ]:
select * from docs_categories;


In [ ]:
update docs_chunks_table 
  SET category = docs_categories.category
  from docs_categories
  where  docs_chunks_table.relative_path = docs_categories.relative_path;


### Images

Now let's process the images we have for our bikes and skis. We are going to use COMPLETE multi-modal function to generate image descriptions and classifications. We will add this information to the DOCS_CHUNKS_TABLE where we also have the PDF documentation.

In [ ]:
insert into DOCS_CHUNKS_TABLE (relative_path, chunk, chunk_index, category)
SELECT 
    RELATIVE_PATH,
    CONCAT('This is a picture describing the bike: '|| RELATIVE_PATH || 
        'THIS IS THE DESCRIPTION: ' ||
        SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet',
        'DESCRIBE THIS IMAGE: ',
        TO_FILE('@DOCS', RELATIVE_PATH))) as chunk,
    0,
    SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet',
        'Classify this image, respond only with Bike or Snow: ',
        TO_FILE('@DOCS', RELATIVE_PATH)) as category,
FROM 
    DIRECTORY('@DOCS')
WHERE
    RELATIVE_PATH LIKE '%.jpeg';


In [ ]:
select * from DOCS_CHUNKS_TABLE
    where RELATIVE_PATH LIKE '%.jpeg';

### Cortex Search

Cortex Search tool will be used to retrieve context from unstructured data. Once we have processed all the content from PDFs and images into the DOCS_CHUNK_TABLE, we just need to enable the service in that table. This will automatically create the embeddings, indexing, etc. 

In [ ]:
CREATE WAREHOUSE IF NOT EXISTS DEFAULT_WH
WAREHOUSE_SIZE = 'XSMALL';

In [ ]:
create or replace CORTEX SEARCH SERVICE DOCUMENTATION_TOOL
ON chunk
ATTRIBUTES relative_path, category
warehouse = DEFAULT_WH
TARGET_LAG = '1 hour'
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
as (
    select chunk,
        chunk_index,
        relative_path,
        category
    from docs_chunks_table
);

### Cortex Analyst: Tool for Structured Data

Another tool that we will setup is Cortex Analyst. It will provide the capability to extract information from structured data stored in Snowflake tables. In the API call we will provider the location of our semantic file that contains information about the business terminology used to describe the data.

First, let's create some tables and generate data that provides additional context about bikes and ski products.

**DIM_ARTICLE – Article/Item Dimension**

Purpose: Stores descriptive information about the products (articles) being sold.

Key Columns:

- ARTICLE_ID (Primary Key): Unique identifier for each article.
- ARTICLE_NAME: Full name/description of the product.
- ARTICLE_CATEGORY: Product category (e.g., Bike, Skis, Ski Boots).
- ARTICLE_BRAND: Manufacturer or brand (e.g., Mondracer, Carver).
- ARTICLE_COLOR: Dominant color for the article.
- ARTICLE_PRICE: Standard unit price of the article.


**DIM_CUSTOMER – Customer Dimension**

Purpose: Contains demographic and segmentation info about each customer.

Key Columns:

- CUSTOMER_ID (Primary Key): Unique identifier for each customer.
- CUSTOMER_NAME: Display name for the customer.
- CUSTOMER_REGION: Geographic region (e.g., North, South).
- CUSTOMER_AGE: Age of the customer.
- CUSTOMER_GENDER: Gender (Male/Female).
- CUSTOMER_SEGMENT: Marketing segment (e.g., Premium, Regular, Occasional).


**FACT_SALES – Sales Transactions Fact Table**

Purpose: Captures individual sales transactions (facts) with references to article and customer details.

Key Columns:

- SALE_ID (Primary Key): Unique identifier for the transaction.
- ARTICLE_ID (Foreign Key): Links to DIM_ARTICLE.
- CUSTOMER_ID (Foreign Key): Links to DIM_CUSTOMER.
- DATE_SALES: Date when the sale occurred.
- QUANTITY_SOLD: Number of units sold in the transaction.
- TOTAL_PRICE: Total transaction value (unit price × quantity).
- SALES_CHANNEL: Sales channel used (e.g., Online, In-Store, Partner).
- PROMOTION_APPLIED: Boolean indicating if the sale involved a promotion or discount.



In [ ]:
CREATE OR REPLACE TABLE DIM_ARTICLE (
    ARTICLE_ID INT PRIMARY KEY,
    ARTICLE_NAME STRING,
    ARTICLE_CATEGORY STRING,
    ARTICLE_BRAND STRING,
    ARTICLE_COLOR STRING,
    ARTICLE_PRICE FLOAT
);

INSERT INTO DIM_ARTICLE (ARTICLE_ID, ARTICLE_NAME, ARTICLE_CATEGORY, ARTICLE_BRAND, ARTICLE_COLOR, ARTICLE_PRICE)
VALUES 
(1, 'Mondracer Infant Bike', 'Bike', 'Mondracer', 'Red', 3000),
(2, 'Premium Bicycle', 'Bike', 'Veloci', 'Blue', 9000),
(3, 'Ski Boots TDBootz Special', 'Ski Boots', 'TDBootz', 'Black', 600),
(4, 'The Ultimate Downhill Bike', 'Bike', 'Graviton', 'Green', 10000),
(5, 'The Xtreme Road Bike 105 SL', 'Bike', 'Xtreme', 'White', 8500),
(6, 'Carver Skis', 'Skis', 'Carver', 'Orange', 790),
(7, 'Outpiste Skis', 'Skis', 'Outpiste', 'Yellow', 900),
(8, 'Racing Fast Skis', 'Skis', 'RacerX', 'Blue', 950);

In [ ]:
CREATE OR REPLACE TABLE DIM_CUSTOMER (
    CUSTOMER_ID INT PRIMARY KEY,
    CUSTOMER_NAME STRING,
    CUSTOMER_REGION STRING,
    CUSTOMER_AGE INT,
    CUSTOMER_GENDER STRING,
    CUSTOMER_SEGMENT STRING
);

INSERT INTO DIM_CUSTOMER (CUSTOMER_ID, CUSTOMER_NAME, CUSTOMER_REGION, CUSTOMER_AGE, CUSTOMER_GENDER, CUSTOMER_SEGMENT)
SELECT 
    SEQ4() AS CUSTOMER_ID,
    'Customer ' || SEQ4() AS CUSTOMER_NAME,
    CASE MOD(SEQ4(), 5)
        WHEN 0 THEN 'North'
        WHEN 1 THEN 'South'
        WHEN 2 THEN 'East'
        WHEN 3 THEN 'West'
        ELSE 'Central'
    END AS CUSTOMER_REGION,
    UNIFORM(18, 65, RANDOM()) AS CUSTOMER_AGE,
    CASE MOD(SEQ4(), 2)
        WHEN 0 THEN 'Male'
        ELSE 'Female'
    END AS CUSTOMER_GENDER,
    CASE MOD(SEQ4(), 3)
        WHEN 0 THEN 'Premium'
        WHEN 1 THEN 'Regular'
        ELSE 'Occasional'
    END AS CUSTOMER_SEGMENT
FROM TABLE(GENERATOR(ROWCOUNT => 5000));

In [ ]:
CREATE OR REPLACE TABLE FACT_SALES (
    SALE_ID INT PRIMARY KEY,
    ARTICLE_ID INT,
    DATE_SALES DATE,
    CUSTOMER_ID INT,
    QUANTITY_SOLD INT,
    TOTAL_PRICE FLOAT,
    SALES_CHANNEL STRING,
    PROMOTION_APPLIED BOOLEAN,
    FOREIGN KEY (ARTICLE_ID) REFERENCES DIM_ARTICLE(ARTICLE_ID),
    FOREIGN KEY (CUSTOMER_ID) REFERENCES DIM_CUSTOMER(CUSTOMER_ID)
);

-- Populating Sales Fact Table with new attributes
INSERT INTO FACT_SALES (SALE_ID, ARTICLE_ID, DATE_SALES, CUSTOMER_ID, QUANTITY_SOLD, TOTAL_PRICE, SALES_CHANNEL, PROMOTION_APPLIED)
SELECT 
    SEQ4() AS SALE_ID,
    A.ARTICLE_ID,
    DATEADD(DAY, UNIFORM(-1095, 0, RANDOM()), CURRENT_DATE) AS DATE_SALES,
    UNIFORM(1, 5000, RANDOM()) AS CUSTOMER_ID,
    UNIFORM(1, 10, RANDOM()) AS QUANTITY_SOLD,
    UNIFORM(1, 10, RANDOM()) * A.ARTICLE_PRICE AS TOTAL_PRICE,
    CASE MOD(SEQ4(), 3)
        WHEN 0 THEN 'Online'
        WHEN 1 THEN 'In-Store'
        ELSE 'Partner'
    END AS SALES_CHANNEL,
    CASE MOD(SEQ4(), 4)
        WHEN 0 THEN TRUE
        ELSE FALSE
    END AS PROMOTION_APPLIED
FROM DIM_ARTICLE A
JOIN TABLE(GENERATOR(ROWCOUNT => 10000)) ON TRUE
ORDER BY DATE_SALES;

### Semantic Models

The semantic model maps business terminology to the structured data and adds contextual meaning. It allows Cortex Analyst to generate the correct SQL for a question asked in natural language.

NOTE: To explore semantic models, refer to these [instructions](https://github.com/Snowflake-Labs/sfguide-build-data-agents-using-snowflake-cortex-ai?tab=readme-ov-file#step-5-explore-the-semantic-model).

In [ ]:
create or replace stage semantic_files ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );

COPY FILES
    INTO @semantic_files/
    FROM @DASH_CORTEX_AGENTS_SUMMIT.PUBLIC.git_repo/branches/main/
    FILES = ('semantic.yaml', 'semantic_search.yaml');

### Improve Tool Usage with Dynamic Literal Retrieval

Using Cortex Analyst integration with Cortex Search, we can improve the retrieval of possible values of a column without listing them all in the semantic model file. 

Let's try it as example for the ARTICLE NAMES.

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE _ARTICLE_NAME_SEARCH
  ON ARTICLE_NAME
  WAREHOUSE = DEFAULT_WH
  TARGET_LAG = '1 hour'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT ARTICLE_NAME
  FROM DIM_ARTICLE
);

### *NOTE: This concludes the setup tools portion of this guide. Follow [instructions outlined here](https://github.com/Snowflake-Labs/sfguide-build-data-agents-using-snowflake-cortex-ai?tab=readme-ov-file#important-note-run-through-all-the-cells-in-the-snowflake-notebook-before-proceeding) to complete the lab.*